# Pandas Workflow com LlamaIndex Workflows

>Este notebook demonstra como criar um sistema de análise de dados usando LlamaIndex, Pandas e Groq para consultas em linguagem natural.


## Sobre a Atualização do LlamaIndex

O **LlamaIndex** descontinuou o módulo **QueryPipeline** em favor de uma nova abordagem chamada **Workflows**.
Essa mudança foi implementada na versão `0.11` do LlamaIndex, com o objetivo de oferecer uma arquitetura mais
flexível e escalável para a construção de aplicações de IA generativa.

### Por que o QueryPipeline foi descontinuado?
O QueryPipeline era uma API declarativa útil para orquestrar consultas simples a avançadas sobre os dados.
Porém, ele se mostrou limitado para cenários mais dinâmicos. Por isso, os **Workflows** foram introduzidos,
trazendo uma arquitetura orientada a eventos para fluxos de trabalho mais sofisticados.

### O que são Workflows?
Workflows permitem orquestrar etapas personalizadas de forma assíncrona e condicional, facilitando
integrações complexas e manipulação de dados em tempo real.

**Documentação oficial:** [docs.llamaindex.ai](https://docs.llamaindex.ai)



#Código:


##1 Instalar Dependências:

In [1]:
# Instalar dependências (-q para instalação silenciosa)
!pip install -q jedi>=0.16 llama-index llama-index-llms-openai llama-index-llms-groq llama-index-experimental gradio fpdf

##1.1 Para uso no Colab


In [2]:
import nest_asyncio
nest_asyncio.apply()

##2 Imports

In [39]:
import pandas as pd
import textwrap
import re
import os
import requests
from pydantic import BaseModel, Field, field_validator, ConfigDict, ValidationError
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.core.workflow import Workflow, Event, StartEvent as BaseStartEvent, StopEvent, step
from llama_index.core.prompts import PromptTemplate
from google.colab import userdata
from typing import Any, List, Tuple, Optional
import asyncio
import matplotlib.font_manager as fm
import gradio as gr
from fpdf import FPDF
from datetime import datetime

##3 Instruções e Prompts

(Importante para validar a saída, pode ser usado como debug ou  )

In [4]:
# ===== INSTRUÇÕES PARA CONVERSÃO DE CONSULTAS =====
instruction_str = (
    "1. Converta a consulta para código Python executável usando Pandas.\n"
    "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
    "3. O código deve representar uma solução para a consulta.\n"
    "4. IMPRIMA APENAS A EXPRESSÃO FINAL.\n"
    "5. Não coloque a expressão entre aspas.\n"
    "6. Evite atribuições (=) na linha final - prefira expressões que retornem valores.\n"
    "7. Para operações de múltiplas linhas, termine com uma expressão que retorne o resultado.\n"
    "8. Exemplos válidos:\n"
    "   - df.groupby('coluna')['valor'].sum()\n"
    "   - df['coluna'].value_counts().head(5)\n"
    "   - df.describe()\n"
    "9. Evite códigos como 'df['coluna'] = valor' - prefira consultas que retornem dados.\n"
)

# ===== PROMPTS =====
pandas_prompt_str = (
    "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
    "{colunas_detalhes}\n\n"
    "Este é o resultado de `print(df.head())`:\n"
    "{df_str}\n\n"
    "Siga estas instruções:\n"
    "{instruction_str}\n"
    "Consulta: {query_str}\n\n"
    "Expressão:"
)

RESPONSE_SYNTHESIS_PROMPT_STR = (
   "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta.\n"
   "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
   "Consulta: {query_str}\n\n"
   "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
   "Saída do Pandas: {pandas_output}\n\n"
   "Resposta: "
   "Ao final, exibir o código usado para gerar a resposta, no formato: O código utilizado foi {pandas_instructions}"
)

print("✅ Prompts configurados com sucesso!")

✅ Prompts configurados com sucesso!


##4 Configuração Pydantic V2

In [5]:
class LLMConfig(BaseModel):
    model: str = Field(..., description="Nome do modelo Groq a ser usado")
    api_key: str = Field(..., description="Chave da API Groq")
    data_url: str = Field(..., description="URL do CSV com os dados")

    @field_validator("data_url")
    @classmethod
    def validar_url(cls, v: str) -> str:
        if not (v.startswith("http://") or v.startswith("https://")):
            raise ValueError("data_url deve começar com http:// ou https://")
        return v

    @field_validator("api_key")
    @classmethod
    def validar_api_key(cls, v: str) -> str:
        if not v or len(v.strip()) == 0:
            raise ValueError("api_key não pode ser vazia")
        return v

print("✅ Configuração LLM com Pydantic criada!")

class Historico(BaseModel):
    entradas: List[Tuple[str, str]] = []

    def adicionar(self, pergunta: str, resposta: str):
        """Adiciona uma entrada ao histórico se pergunta e resposta forem válidas."""
        if pergunta and resposta:
            self.entradas.append((pergunta, resposta))

print("✅ Modelo de histórico com Pydantic criado!")

✅ Configuração LLM com Pydantic criada!
✅ Modelo de histórico com Pydantic criado!


##5 Modelos de Eventos

(Rastrear cada passo da interação, como um histórico organizado)

In [6]:
# --- Modelos de Eventos com Pydantic ---

# --- Evento de início customizado para garantir os dados de entrada ---
class StartEvent(BaseStartEvent):
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

# --- Eventos intermediários para propagar os dados no workflow ---
class CodeEvent(Event):
    pandas_prompt: str
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class OutputEvent(Event):
    pandas_code: str
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

class ExecutedEvent(Event):
    pandas_code: str
    pandas_output: Any
    query: str
    df: pd.DataFrame
    model_config = ConfigDict(arbitrary_types_allowed=True)

print("✅ Modelos de eventos do workflow definidos com sucesso!")

✅ Modelos de eventos do workflow definidos com sucesso!


##6 Funções Auxiliares

In [7]:
def carregar_arquivo(caminho_arquivo: str, df_estado: Optional[pd.DataFrame] = None) -> Tuple[str, Optional[pd.DataFrame]]:
    """
    Carrega um CSV do caminho fornecido. Se caminho_arquivo for None, mantém df_estado.
    Retorna mensagem de status e DataFrame atualizado.
    """
    if not caminho_arquivo:
        return "Nenhum arquivo enviado. Usando dados atuais (se houver).", df_estado
    try:
        df_novo = pd.read_csv(caminho_arquivo)
        return "Arquivo carregado com sucesso!", df_novo
    except Exception as e:
        return f"Erro ao carregar arquivo: {str(e)}", df_estado

def formatar_texto(response: str, largura: int = 100, imprimir: bool = True):
    texto_formatado = textwrap.fill(response, width=largura)
    if imprimir:
        print(texto_formatado)
    return texto_formatado

def descricao_colunas(df: pd.DataFrame) -> str:
    descricao = "\n".join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Colunas do DataFrame:\n" + descricao

def limpar_codigo_pandas(codigo: str) -> str:
    """Limpa e valida o código Pandas gerado pela LLM."""
    codigo = re.sub(r'```(?:python)?\n?', '', codigo) # Remove ```python
    codigo = re.sub(r'```', '', codigo) # Remove ``` de fechamento
    linhas = [linha.strip() for linha in codigo.split('\n') if linha.strip()]

    # Filtra linhas que são comentários ou texto explicativo
    codigo_filtrado = [
        linha for linha in linhas if not (
            linha.startswith('#') or
            'resposta:' in linha.lower() or
            'resultado:' in linha.lower()
        )
    ]

    if not codigo_filtrado:
        return ""

    # Se houver várias linhas, a última é geralmente a expressão de retorno
    return codigo_filtrado[-1]

print("✅ Funções auxiliares configuradas!")

✅ Funções auxiliares configuradas!


##7 Configuração da API e Dados

In [8]:
# Configure sua chave da API Groq nos Secrets do Colab
# Vá em:  (ícone da chave) -> Add new secret
# Name: Groq_API
# Value: sua_chave_aqui

try:
    key = userdata.get("Groq_API")
    print("✅ Chave API carregada com sucesso!")
except Exception as e:
    key = None
    print(f"❌ Erro ao carregar chave API: {e}. Por favor, configure a chave 'Groq_API' nos Secrets do Colab.")

# ===== CONFIGURAÇÃO E DADOS =====
if key:
    config = LLMConfig(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        api_key=key,
        data_url="https://raw.githubusercontent.com/YuriArduino/Estudos_Artificial_Intelligence/refs/heads/Dados/vendas.csv"
    )

    df = pd.read_csv(config.data_url)
    Settings.llm = Groq(model=config.model, api_key=config.api_key)

    print("\n Estrutura do DataFrame:")
    print(df.head())
    print(f"\n✅ Configuração completa! Dataset: {df.shape[0]} linhas, {df.shape[1]} colunas")
else:
    df = None
    print("\n⚠️ A execução será interrompida pois a API Key não foi carregada.")

✅ Chave API carregada com sucesso!

 Estrutura do DataFrame:
     ID_compra filial       cidade tipo_cliente     genero       tipo_produto  \
0  750-67-8428      A  Santo André       Membro   Feminino     Saúde e Beleza   
1  226-31-3081      C  São Caetano       Normal   Feminino        Eletrônicos   
2  631-41-3108      A  Santo André       Normal  Masculino               Casa   
3  123-19-1176      A  Santo André       Membro  Masculino     Saúde e Beleza   
4  373-73-7910      A  Santo André       Normal  Masculino  Esportes e Viagem   

   preco_unitario  quantidade  imposto_5%     total        data      hora  \
0           74.69           7     26.1415  548.9715  2024-01-05  13:08:00   
1           15.28           5      3.8200   80.2200  2024-03-08  10:29:00   
2           46.33           7     16.2155  340.5255  2024-03-03  13:23:00   
3           58.22           8     23.2880  489.0480  2024-01-27  20:33:00   
4           86.31           7     30.2085  634.3785  2024-02-08  10

##8 Classe Workflow Principal

In [9]:
class PandasWorkflow(Workflow):

    @step
    async def iniciar_processamento(self, ev: StartEvent) -> CodeEvent:
        """
        Prepara o prompt para gerar o código Pandas.
        Os dados 'query' e 'df' são recebidos diretamente do StartEvent.
        """
        print("  [Workflow] Etapa 1: Iniciando processamento...")
        colunas_info = descricao_colunas(ev.df)
        prompt_text = pandas_prompt_str.format(
            colunas_detalhes=colunas_info,
            df_str=ev.df.head(5).to_string(),
            instruction_str=instruction_str,
            query_str=ev.query
        )
        return CodeEvent(pandas_prompt=prompt_text, query=ev.query, df=ev.df)

    @step
    async def gerar_codigo(self, ev: CodeEvent) -> OutputEvent:
        """Gera código pandas a partir do prompt."""
        print("  [Workflow] Etapa 2: Gerando código Pandas...")
        response = await Settings.llm.acomplete(ev.pandas_prompt)
        codigo_limpo = limpar_codigo_pandas(str(response).strip())
        print(f"   ✅ Código gerado: {codigo_limpo}")
        return OutputEvent(pandas_code=codigo_limpo, query=ev.query, df=ev.df)

    @step
    async def executar_codigo(self, ev: OutputEvent) -> ExecutedEvent:
        """Executa o código Pandas gerado."""
        print(" [Workflow] Etapa 3: Executando código...")
        try:
            # Contexto seguro para a execução do eval
            contexto = {"df": ev.df, "pd": pd}
            resultado = eval(ev.pandas_code, {"__builtins__": {}}, contexto)
            print(f"   ✅ Resultado da execução: {resultado}")
        except Exception as e:
            print(f"   ❌ Erro na execução: {e}")
            resultado = f"Erro ao executar o código: {str(e)}"
        return ExecutedEvent(
            pandas_code=ev.pandas_code,
            pandas_output=resultado,
            query=ev.query,
            df=ev.df
        )

    @step
    async def finalizar_e_sintetizar(self, ev: ExecutedEvent) -> StopEvent:
        """Gera a resposta final em linguagem natural."""
        print("  [Workflow] Etapa 4: Sintetizando resposta final...")
        if isinstance(ev.pandas_output, str) and "Erro" in ev.pandas_output:
            resposta_final = f"Não foi possível processar a consulta devido a um erro. {ev.pandas_output}"
        else:
            prompt_synthesis = RESPONSE_SYNTHESIS_PROMPT_STR.format(
                query_str=ev.query,
                pandas_instructions=ev.pandas_code,
                pandas_output=str(ev.pandas_output)
            )
            response = await Settings.llm.acomplete(prompt_synthesis)
            resposta_final = str(response).strip()

        print(" [Workflow] Finalizado com sucesso!")
        return StopEvent(result={
            "resposta_final": resposta_final,
            "pandas_code": ev.pandas_code,
        })

print("✅ Workflow robusto e simplificado definido!")

✅ Workflow robusto e simplificado definido!


##9 Função de Execução

In [23]:
async def executar_consulta(query: str, df_local: pd.DataFrame):
    """
    Executa o workflow de ponta a ponta e trata os erros de forma explícita, sem fallback.
    """
    if df_local is None:
        return {
            "resposta_final": "Erro: O DataFrame não foi carregado.",
            "pandas_code": "N/A"
        }
    if not query or not query.strip():
        return {
            "resposta_final": "Erro: A consulta não pode ser vazia.",
            "pandas_code": "N/A"
        }

    try:
        print(f"\n Iniciando workflow para a consulta: '{query}'")
        wf = PandasWorkflow()

        # O método run() está retornando o dicionário diretamente.
        # A correção é tratar o resultado como o dicionário final.
        resultado_final = await wf.run(query=query, df=df_local)

        # Verificamos se o resultado é de fato um dicionário, como esperado.
        if not isinstance(resultado_final, dict):
             raise TypeError(f"O workflow retornou um tipo inesperado: {type(resultado_final)}")

        return resultado_final

    except Exception as e:
        # SEM FALLBACK: Se o workflow falhar, reportamos o erro diretamente.
        print(f"❌ Erro crítico durante a execução do workflow: {e}")
        import traceback
        traceback.print_exc() # Imprime o stack trace completo para depuração
        return {
            "resposta_final": f"Ocorreu um erro crítico no workflow. Verifique os logs para detalhes. Erro: {str(e)}",
            "pandas_code": "N/A - Falha no workflow"
        }


print("✅ Função de execução do workflow corrigida: sem fallback e com tratamento de erro explícito!")

✅ Função de execução do workflow corrigida: sem fallback e com tratamento de erro explícito!


##10 Gradio

###10.1 Célula de preparação

In [40]:
# Define uma variável global para o caminho da fonte
FONT_PATH = ""
FONT_NAME_TO_FIND = "DejaVu Sans"

try:
    # A maneira mais robusta: usar o gerenciador de fontes do Matplotlib
    print(f"Procurando a fonte '{FONT_NAME_TO_FIND}' localmente com Matplotlib...")
    FONT_PATH = fm.findfont(FONT_NAME_TO_FIND, fallback_to_default=True)

    if not os.path.exists(FONT_PATH):
        raise FileNotFoundError  # Força a entrada no bloco de exceção se o caminho for inválido

    print(f"✅ Fonte encontrada com sucesso em: {FONT_PATH}")

except Exception:
    print(f"⚠️ Fonte '{FONT_NAME_TO_FIND}' não encontrada. Tentando baixar uma alternativa confiável (Roboto)...")

    # --- PLANO B: Se o Matplotlib falhar, baixa uma fonte de um link estável ---
    FONT_FILE_NAME = "Roboto-Regular.ttf"
    FONT_URL = "https://github.com/google/fonts/raw/main/ofl/roboto/Roboto-Regular.ttf"

    if not os.path.exists(FONT_FILE_NAME):
        try:
            print(f"Baixando '{FONT_FILE_NAME}'...")
            response = requests.get(FONT_URL)
            response.raise_for_status()
            with open(FONT_FILE_NAME, "wb") as f:
                f.write(response.content)
            FONT_PATH = FONT_FILE_NAME
            print(f"✅ Fonte alternativa baixada com sucesso!")
        except Exception as download_error:
            print(f"❌ FALHA CRÍTICA: Não foi possível encontrar a fonte localmente nem baixá-la. Erro: {download_error}")
            FONT_PATH = ""  # Garante que o caminho esteja vazio em caso de falha
    else:
        FONT_PATH = FONT_FILE_NAME
        print(f"✅ Fonte alternativa '{FONT_FILE_NAME}' já existe localmente.")

Procurando a fonte 'DejaVu Sans' localmente com Matplotlib...
✅ Fonte encontrada com sucesso em: /usr/local/lib/python3.12/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf


###10.2 Funções auxiliares Gradio

In [35]:
def carregar_dados(file, df_estado):
    if file is None:
        return "❌ Nenhum arquivo selecionado", df_estado
    try:
        df_local = pd.read_csv(file.name)
        return f"✅ Arquivo carregado! {df_local.shape[0]} linhas, {df_local.shape[1]} colunas.", df_local
    except Exception as e:
        return f"❌ Erro ao carregar arquivo: {str(e)}", df_estado

def processar_pergunta(pergunta: str, df_local: Optional[pd.DataFrame], historico: Historico):
    if df_local is None:
        resposta = "❌ Por favor, carregue um arquivo CSV primeiro."
        return resposta, historico

    if not pergunta or not pergunta.strip():
        resposta = "❌ Por favor, digite uma pergunta válida."
        return resposta, historico

    try:
        resultado = asyncio.run(executar_consulta(pergunta, df_local))
        resposta = resultado.get("resposta_final", "❌ Erro: a resposta não foi gerada.")
        historico.adicionar(pergunta, resposta)
        return resposta, historico
    except Exception as e:
        erro_msg = f"❌ Ocorreu um erro inesperado: {str(e)}"
        historico.adicionar(pergunta, erro_msg)
        return erro_msg, historico

# --- FUNÇÃO GERAR_PDF ---
# A variável FONT_PATH é definida na célula de preparação anterior.
if 'FONT_PATH' not in globals() or not FONT_PATH:
    print("❌ ERRO: O caminho da fonte (FONT_PATH) não foi definido. Execute a célula de preparação.")
    FONT_PATH = ""

def gerar_pdf(historico: Historico):
    if not historico or not historico.entradas:
        print("⚠️ Tentativa de gerar PDF com histórico vazio.")
        return None

    # Verifica se o caminho da fonte é válido ANTES de tentar gerar o PDF
    if not FONT_PATH or not os.path.exists(FONT_PATH):
        msg = f"❌ Erro ao gerar PDF: Arquivo da fonte não foi encontrado em '{FONT_PATH}'. Execute a célula de preparação."
        print(msg)
        # Opcional: retornar um PDF de erro
        return None

    print(f"Gerando PDF com {len(historico.entradas)} entradas usando a fonte: {FONT_PATH}")

    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        caminho_pdf = f"relatorio_consultas_{timestamp}.pdf"

        pdf = FPDF()
        pdf.add_page()

        # --- MUDANÇA FINAL ---
        # Usa o caminho absoluto da fonte encontrado no sistema.
        pdf.add_font('DejaVu', '', FONT_PATH, uni=True)
        pdf.set_font('DejaVu', '', 10)

        pdf.set_font('DejaVu', '', 16)
        pdf.cell(0, 10, "Relatório de Consultas ao CSV", ln=True, align="C")
        pdf.ln(10)

        for i, (pergunta, resposta) in enumerate(historico.entradas, 1):
            pdf.set_font('DejaVu', '', 12)
            pdf.multi_cell(0, 8, f"Pergunta {i}: {pergunta}")
            pdf.ln(2)
            pdf.set_font('DejaVu', '', 10)
            pdf.multi_cell(0, 6, f"Resposta: {resposta}")
            pdf.ln(8)
            pdf.cell(0, 0, '', 'T')
            pdf.ln(8)

        pdf.output(caminho_pdf)
        print(f"✅ PDF gerado com sucesso: {caminho_pdf}")
        return caminho_pdf

    except Exception as e:
        print(f"❌ Erro crítico ao gerar PDF: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Funções auxiliares do Gradio prontas para usar fonte local.")

# --- FUNÇÃO EXTRA PARA LIMPAR A INTERFACE ---
def limpar_tudo():
    # Retorna valores vazios/padrão para limpar os componentes
    return "", None, Historico()

✅ Funções auxiliares do Gradio prontas para usar fonte local.


###10.3 Interface Gradio

In [62]:
with gr.Blocks(title="Analisador de CSV com Pandas e LlamaIndex", theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1>🐼 Analisador de CSV com Pandas e LlamaIndex</h1>
            <p>Faça upload de um arquivo CSV, faça perguntas em linguagem natural e baixe o relatório!</p>
        </div>
        """
    )

    # Estados
    df_estado = gr.State()
    historico_estado = gr.State(value=Historico())

    with gr.Row():
        with gr.Column(scale=1):
            input_arquivo = gr.File(label="1. Faça upload do seu arquivo CSV", file_types=[".csv"])
            upload_status = gr.Textbox(label="Status do Upload", interactive=False, placeholder="Aguardando arquivo...")
            botao_gerar_pdf = gr.Button("📄 Gerar e Baixar PDF", variant="secondary")
            output_pdf = gr.File(label="Baixar Relatório", interactive=False)

        with gr.Column(scale=3):
            # --- Componente Chatbot ---
            chatbot = gr.Chatbot(
                label="Histórico da Conversa",
                height=550,
                type='messages'
            )
            with gr.Row():
                input_pergunta = gr.Textbox(
                    scale=4,
                    show_label=False,
                    placeholder="Ex: Qual o total de vendas por filial?",
                    container=False
                )
                botao_enviar = gr.Button(" Enviar", variant="primary", scale=1, min_width=100)

    # --- Lógica da Interface ---

    def chat_interface(pergunta, chat_history, df_local, historico_obj):
        # O chat_history agora é uma lista de dicionários.
        # Adiciona a mensagem do usuário ao histórico no formato correto.
        chat_history.append({"role": "user", "content": pergunta})

        if df_local is None:
            resposta = "❌ Por favor, carregue um arquivo CSV primeiro."
            # Adiciona a resposta do assistente ao histórico.
            chat_history.append({"role": "assistant", "content": resposta})
            return "", chat_history, historico_obj

        # Chama a função de processamento principal, que não precisa mudar.
        resposta, historico_obj_atualizado = processar_pergunta(pergunta, df_local, historico_obj)

        # Adiciona a resposta do assistente ao histórico .
        chat_history.append({"role": "assistant", "content": resposta})

        return "", chat_history, historico_obj_atualizado

    # Função para limpar os estados
    def limpar_conversa():
        return Historico(), None

    # --- Conexões dos Eventos ---
    input_arquivo.upload(fn=carregar_dados, inputs=[input_arquivo, df_estado], outputs=[upload_status, df_estado])

    botao_enviar.click(
        fn=chat_interface,
        inputs=[input_pergunta, chatbot, df_estado, historico_estado],
        outputs=[input_pergunta, chatbot, historico_estado]
    )
    input_pergunta.submit(
        fn=chat_interface,
        inputs=[input_pergunta, chatbot, df_estado, historico_estado],
        outputs=[input_pergunta, chatbot, historico_estado]
    )

    botao_gerar_pdf.click(fn=gerar_pdf, inputs=[historico_estado], outputs=[output_pdf])

    chatbot.clear(fn=limpar_conversa, inputs=[], outputs=[historico_estado, output_pdf])

print("✅ Interface Gradio finalizada!")

# Lança a interface
if __name__ == "__main__":
    if df is not None:
        app.launch(debug=True)
    else:
        print("🔴 A interface Gradio não será iniciada pois a configuração da API falhou.")

✅ Interface Gradio finalizada, moderna e sem warnings!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef5393479a158f0e87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ef5393479a158f0e87.gradio.live


##11 Teste Rápido

In [ ]:
# Teste rápido para verificar se a função principal está funcionando
async def teste_rapido():
    if df is not None:
        query_teste = "Qual é a avaliação média de cada filial?"
        # Corrigido: passando o df_local como argumento
        resultado = await executar_consulta(query_teste, df_local=df)

        if resultado and "Erro" not in resultado.get("resposta_final", ""):
            print("\n✅ Sistema funcionando perfeitamente!")
            print(f"Resposta: {resultado['resposta_final']}")
        else:
            print(f"\n❌ Houve um problema no teste: {resultado}")
    else:
        print("⚠️ Teste não executado: DataFrame não foi carregado.")

# Executar o teste
await teste_rapido()

##12 Exemplos de Uso Individual

In [ ]:
# Execute uma consulta por vez para testar

# Exemplo 1
query = "Quais são os 5 produtos mais vendidos?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 2
query = "Qual o total de vendas por região?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 3
query = "Qual produto tem a maior margem de lucro?"
result = await executar_consulta(query)

In [ ]:
# Exemplo 4
query = "Quantas vendas foram feitas em cada mês?"
result = await executar_consulta(query)

In [ ]:
query = "Qual o genêro que mais consome e o segmento do produto?"
result = await executar_consulta(query)

##13 Bateria de Testes

In [ ]:
# Executar múltiplas consultas de uma vez
consultas_exemplo = [
    "Qual é a avaliação média de cada filial?",
    "Quais são os 5 produtos mais vendidos?",
    "Qual o total de vendas por região?",
    "Qual produto tem a maior margem de lucro?",
    "Quantas vendas foram feitas em cada mês?",
    "Qual é a receita total por categoria de produto?",
    "Qual filial teve melhor desempenho?",
    "Mostre a distribuição de preços dos produtos"
]

print(" Executando bateria de testes...")
resultados = await executar_multiplas_consultas(consultas_exemplo)

print(f"\n✅ Concluído! {len([r for r in resultados if r])} consultas processadas com sucesso.")

##14 Modo Interativo S/ interface Gradio

In [ ]:
# Modo interativo - digite suas próprias consultas
print(" MODO INTERATIVO")
print("Digite suas consultas (digite 'sair' para terminar)")
print("-" * 50)

while True:
    try:
        query = input("\n Sua consulta: ").strip()

        if query.lower() in ['sair', 'exit', 'quit', '']:
            print(" Até logo!")
            break

        await executar_consulta(query, mostrar_detalhes=False)

    except KeyboardInterrupt:
        print("\n Interrompido pelo usuário!")
        break
    except Exception as e:
        print(f" Erro: {e}")

##15 Análise Manual dos Dados


In [ ]:
# Análise dos dados originais para comparação
print(" ANÁLISE MANUAL DOS DADOS (para comparação)")
print("="*50)

print(f"Estatísticas básicas:")
print(df.describe())

print(f"\nContagem por categoria:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head())

print(f"\nValores nulos:")
print(df.isnull().sum())

#Dicas e Informações

In [ ]:
print("💡 DICAS PARA USO:")
print("="*50)
print("1. Para consultas complexas, seja específico")
print("2. Use nomes exatos das colunas mostradas na análise")
print("3. Experimente diferentes tipos de análise:")
print("   - Agrupamentos: 'média por categoria'")
print("   - Rankings: 'top 10 produtos'")
print("   - Filtros: 'vendas acima de X valor'")
print("   - Séries temporais: 'vendas por mês'")
print("4. O sistema mostra o código Pandas usado!")

print(f"\n INFORMAÇÕES DO DATASET:")
print(f"Suas colunas: {list(df.columns)}")
print(f"Quantidade de dados: {df.shape[0]} linhas x {df.shape[1]} colunas")
print(f"Tipos de dados: {df.dtypes.value_counts().to_dict()}")

print(f"\n SUGESTÕES DE CONSULTAS:")
example_queries = [
    "Qual a correlação entre preço e quantidade vendida?",
    "Mostre a distribuição de vendas por dia da semana",
    "Qual filial tem o melhor desempenho?",
    "Compare as vendas do primeiro e último trimestre",
    "Identifique produtos com baixo estoque",
    "Qual é o produto mais caro por categoria?",
    "Mostre as vendas médias por vendedor",
    "Qual região tem menor variação de preços?"
]

for i, query in enumerate(example_queries, 1):
    print(f"{i}. {query}")

print(f"\n MODELO UTILIZADO: {config.model}")
print(f" URL DOS DADOS: {config.data_url}")

#Notas de Uso:

*   Configuração da API: Configure sua chave Groq nos Secrets do Colab
*   Execução sequencial: Execute as células na ordem
*   Personalização: Modifique a URL dos dados na Célula 6
*   Debug: Os logs mostram o código Pandas gerado
*   Interativo: Use a Célula 12 para consultas livres

##Estrutura final:

*   Instalação automática de dependências
* Configuração completa da API
* Exploração automática dos dados
* Sistema de workflow robusto
* Exemplos prontos para uso
* Modo interativo
* Sistema de debug integrado


>Desenvolvido por Yuri Arduino Bernardineli Alves | GitHub: [YuriArduino](https://github.com/YuriArduino) | E-mail: yuriarduino@gmail.com